In [13]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import utils

In [15]:
data = utils.load_data('../data/output.csv')

In [16]:
data.sample(5)

,Evento,Puerta,FechaHora,Rol
32803,Acceso concedido,ML-VEHICULAR-ML-T11,2024/10/01 20:52:07,ESTUDIANTE
31311,Acceso concedido,ML-VEHICULAR-ML-T5,2024/10/01 05:23:59,ESTUDIANTE
6225,Acceso concedido,CD7-MOL4-IN-T1,2024/10/01 12:53:28,ASEO
6056,Acceso concedido,ML-VEHICULAR-ML-T2,2024/10/01 05:20:17,PROFESOR
58144,Acceso concedido,CD3-MOL6-IN-T7,2024/10/01 07:17:35,EMPLEADO


In [17]:
data['custom_index'] = (data['Rol']).apply(utils.create_index)
data = data.sort_values(by='FechaHora').reset_index().drop(columns='index')
data.head(100000)

,Evento,Puerta,FechaHora,Rol,custom_index
0,Acceso concedido,SD-VEHICULAR-SD-T5,2024/10/01 00:00:05,ASEO,16857
1,Acceso concedido,ML7-MOL3-IN-T5,2024/10/01 00:00:09,VIGILANCIA,19087
2,Acceso concedido,CD9-MOL4-OUT-T7,2024/10/01 00:00:25,VIGILANCIA,19625
3,Acceso concedido,RGD6-MOL1-OUT-T11,2024/10/01 00:00:25,ASEO,16496
4,Acceso concedido,ML-VEHICULAR-ML-T7,2024/10/01 00:00:34,ASEO,16283
...,...,...,...,...,...
99995,Acceso concedido,ML-VEHICULAR-ML-T10,2024/10/01 23:59:54,PROFESOR,4061
99996,Acceso concedido,CD9-MOL4-OUT-T10,2024/10/01 23:59:54,PROFESOR,4705
99997,Acceso concedido,SD11-MOL1-OUT-T7,2024/10/01 23:59:56,EMPLEADO,18110
99998,Acceso concedido,SD-VEHICULAR-SD-T6,2024/10/01 23:59:57,EMPLEADO,18410


In [18]:
import cv2
import numpy as np
import pytesseract

# Configurar la ruta de Tesseract-OCR (asegúrate de que esté instalado)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # For Windows

# Cargar las imágenes
images = [
    cv2.imread('../assets/Piso1.png'),
    cv2.imread('../assets/Piso2.png'),
    cv2.imread('../assets/Piso5.png'),
    cv2.imread('../assets/Piso6.png'),
]

for i, image in enumerate(images):

    # Convertir la imagen a espacio de color HSV para facilitar la detección de colores
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir los límites del color rojo en el espacio HSV
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Crear máscaras para ambos rangos de rojo
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Unir las dos máscaras para cubrir todas las tonalidades de rojo
    mask = mask1 + mask2

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una lista para almacenar las coordenadas de los cuadrados
    square_coords = []

    # Filtrar solo los contornos que sean cuadrados y que tengan un área significativa
    for contour in contours:
        # Aproximar el contorno
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Verificar si el contorno tiene 4 lados (es un cuadrado o rectángulo)
        if len(approx) == 4 and cv2.contourArea(contour) > 100:  # Filtrar por área
            square_coords.append(approx)

            # Extraer la región del cuadrado detectado de la imagen original
            x, y, w, h = cv2.boundingRect(approx)
            square_roi = image[y:y + h, x:x + w]

            # Preprocesar la imagen antes de OCR (convertir a gris y aplicar umbral)
            gray_square = cv2.cvtColor(square_roi, cv2.COLOR_BGR2GRAY)

            # Binarizar la imagen (solo si es necesario)
            _, threshold_square = cv2.threshold(gray_square, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Usar OCR para leer el texto dentro del cuadrado (configurado para una línea)
            text = pytesseract.image_to_string(threshold_square, config='--psm 7')  # psm 7 para una línea de texto

            # Decidir el color basado en el texto
            if "WAIRA" in text.upper():
                color = (0, 255, 0)  # Verde si el texto es "APPROVED"
            elif "ML-603" in text.upper():
                color = (0, 0, 255)  # Rojo si el texto es "REJECTED"
            else:
                color = (255, 255, 0)  # Amarillo para otros casos

            # Dibujar el cuadrado completo con el color basado en el texto
            cv2.fillPoly(image, [approx], color)

            # Poner el texto extraído en la imagen
            center_x = x + w // 2
            center_y = y + h // 2

            text_size = cv2.getTextSize(text.strip(), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_width, text_height = text_size
            text_x = center_x - text_width // 2
            text_y = center_y + text_height // 2
            text_color = (255, 255, 255)
            cv2.putText(image, text.strip(), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2, text_color, 5, cv2.LINE_AA)

    # Guardar la imagen con los cuadrados detectados y coloreados
    cv2.imwrite(f'../assets/out/{i}.jpg', image)




    # # Imprimir las coordenadas de las esquinas de los cuadrados detectados
    # for i, square in enumerate(square_coords):
    #     print(f'Cuadrado {i+1}:')
    #     for point in square:
    #         x, y = point[0]
    #         print(f'    Esquina: (x: {x}, y: {y})')


## Procesado de datos Oferta de cursos

In [2]:
#%pip install polars
import polars as pl
from ast import literal_eval

In [3]:
df = pl.read_csv('../../scrap/formatted_data.csv', infer_schema_length=10000)

In [4]:
df.sample(5)

Course Code,Course Name,Cupo,NRC,Sección,Créditos,Periodo,Parte de Periodo,Campus,Schedule Table,Cupos Ofrecidos,Cupos Inscritos,Cupos Disponibles
str,str,str,i64,str,i64,i64,str,str,str,i64,i64,i64
"""MATE-1214C""","""CALCULO INTEGRAL CON ECUACIONE…","""2""",72900,"""D3""",0,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'I', 'Hora': '1100-11…",26,24,2
"""BIOL-1327""","""ECOLOGÍA: PRINCIPIOS Y APLICAC…","""3""",57295,"""B""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': 'J', 'Sa…",64,61,3
"""CPER-1201""","""TALLER SONORO (CICLO 1 DE 8 SE…","""1""",36662,"""01""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': 'J', 'Sa…",20,19,1
"""LENG-1158""","""ENGLISH 08 - WRITING 1 (CICLO …","""5""",65922,"""22""",0,202420,"""SEGUNDO CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': 'I', 'Sa…",25,20,5
"""ARQT-3214""","""PROYECTO UNIDAD PROFUNDIZACIÓN…","""Sin cupo""",55501,"""01""",6,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': 'J', 'Sa…",20,20,0


In [5]:
# change the column type to list
df = df.with_columns(
    pl.col("Schedule Table").map_elements(literal_eval).alias("Schedule Table")
)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [6]:
df.sample(10)

Course Code,Course Name,Cupo,NRC,Sección,Créditos,Periodo,Parte de Periodo,Campus,Schedule Table,Cupos Ofrecidos,Cupos Inscritos,Cupos Disponibles
str,str,str,i64,str,i64,i64,str,str,list[struct[6]],i64,i64,i64
"""LENG-1159""","""ENGLISH 09A WRITING 2 - FOR SC…","""4""",54605,"""21""",0,202420,"""SEGUNDO CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""L"",""I"",""V"",""1400-1520"","".AU_402"",""2024-10-07 2024-11-30 .Edif. Aulas""}]",25,21,4
"""LENG-1152""","""ENGLISH 02 (CICLO 1 DE 8 SEMAN…","""4""",39349,"""01""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""L"",""1230-1350"","".SD_403"",""2024-08-05"",""2024-09-28"","".Edif. J.M.Santodomingo (SD)""}, {""M"",""I"",""J"",""1230-1350"","".ML_208"",""2024-08-05 2024-09-28 .Edif. Mario Laserna (ML)""}, {""V"",""1230-1350"","".SD_303"",""2024-08-05"",""2024-09-28"","".Edif. J.M.Santodomingo (SD)""}]",25,21,4
"""ARTE-3609""","""PROYECTO FINAL""","""No info""",66971,"""02""",8,202420,"""PERIODO NO RETIRABLE""","""CAMPUS PRINCIPAL""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",0,1,-1
"""ISIS-3991""","""PRACTICA PROFESIONAL""","""16""",55595,"""01""",6,202420,"""PERIODO NO RETIRABLE""","""CAMPUS PRINCIPAL""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",45,29,16
"""ISIS-2203""","""INFRAESTRUCTURA COMPUTACIONAL""","""2""",25387,"""01""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""L"",""I"",""0930-1050"","".AU_101"",""2024-08-05"",""2024-11-30 .Edif. Aulas""}, {""S"",""0930-1050"","".ML_109"",""2024-08-05"",""2024-11-30"","".Edif. Mario Laserna (ML)""}]",36,34,2
"""LENG-1155""","""ENGLISH 05 (CICLO 1 DE 8 SEMAN…","""2""",42169,"""12""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""L"",""M"",""I"",""J"",""V"",""1230-1350 .AU_401 2024-08-05 2024-09-28 .Edif. Aulas""}]",25,23,2
"""MEDI-3413Q""","""ORTOPEDÍA CIRUGÍA""","""Sin cupo""",69824,"""13""",0,202420,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""M"",""J"",""0700-1250"","".FSFB"",""2024-09-23"",""2024-09-29 .FUNDACION SANTAFE DE BOGOTA""}, {""I"",""0700-1550"","".FSFB"",""2024-09-23"",""2024-09-29"","".FUNDACION SANTAFE DE BOGOTA""}, {""V"",""D"",""0700-1220"","".FSFB"",""2024-09-23"",""2024-09-29 .FUNDACION SANTAFE DE BOGOTA""}]",4,4,0
"""GEOC-3105""","""PETROLOGÍA (CICLO 2 DE 8 SEMAN…","""21""",56884,"""01""",2,202420,"""SEGUNDO CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""J"",""1400-1550"","".O_403"",""2024-10-07"",""2024-11-30 .Edif. Henry Yerly (O)""}]",30,9,21
"""DERE-3101""","""HERMENEUTICA JURIDICA""","""Sin cupo""",69235,"""01""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""V"",""0930-1050"","".O_302"",""2024-08-05"",""2024-11-30 .Edif. Henry Yerly (O)""}]",31,42,-11


In [7]:
#Drop the columns which are not needed
df = df.drop("Cupo", "Sección", "Créditos", "Periodo","Cupos Ofrecidos", "Cupos Disponibles", "NRC","Course Name")

In [8]:
df_virt = pl.DataFrame()
df_virt = df.filter(pl.col('Campus').is_in(["NO DEFINIDO","VIRTUAL","EXTERNO UNIVERSIDAD" ] ))
df_virt.sample(10)

Course Code,Parte de Periodo,Campus,Schedule Table,Cupos Inscritos
str,str,str,list[struct[6]],i64
"""MECA-4405""","""PERIODO 202420 - 16 SEMANAS""","""VIRTUAL""","[{""M"",""J"",""1800-1950"","".VIRT"",""2024-08-05"",""2024-10-26 .VIRTUAL""}]",35
"""MEDI-3407C""","""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".IOIR"",""2024-10-15 2024-10-27 .INSTITUTO ORTOPEDIA INFANTIL R""}, {""M"",""V"",""0700-1150"","".IOIR"",""2024-10-15"",""2024-10-27 .INSTITUTO ORTOPEDIA INFANTIL R""}, {""D"",""0700-1850"","".IOIR"",""2024-10-15"",""2024-10-27"","".INSTITUTO ORTOPEDIA INFANTIL R""}]",4
"""HIST-6985""","""PERIODO NO RETIRABLE""","""NO DEFINIDO""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",1
"""DADM-6982""","""PERIODO 202420 - 16 SEMANAS""","""VIRTUAL""","[{""-"","".VIRT"",""2024-08-05"",""2024-11-30"","".VIRTUAL"",""No info""}]",2
"""MEDI-3406H""","""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".FSFB"",""2024-09-09 2024-09-15 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""V"",""0700-1150"","".FSFB"",""2024-09-09"",""2024-09-15 .FUNDACION SANTAFE DE BOGOTA""}, {""D"",""0700-1850"","".FSFB"",""2024-09-09"",""2024-09-15"","".FUNDACION SANTAFE DE BOGOTA""}]",4
"""MEDI-3406H""","""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""M"",""V"",""0700-1150"","".FSFB"",""2024-11-12"",""2024-11-17 .FUNDACION SANTAFE DE BOGOTA""}, {""I"",""J"",""0700-1550"","".FSFB"",""2024-11-12"",""2024-11-17 .FUNDACION SANTAFE DE BOGOTA""}]",4
"""MEDI-3406G""","""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".FSFB"",""2024-10-21 2024-10-27 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""V"",""0700-1150"","".FSFB"",""2024-10-21"",""2024-10-27 .FUNDACION SANTAFE DE BOGOTA""}, {""0700-1850"","".FSFB"",""2024-10-21"",""2024-10-27"","".FUNDACION"",""SANTAFE DE BOGOTA""}]",4
"""MEDI-3406D""","""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".FSFB"",""2024-09-09 2024-09-22 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""V"",""0700-1150"","".FSFB"",""2024-09-09"",""2024-09-22 .FUNDACION SANTAFE DE BOGOTA""}, {""D"",""0700-1850"","".FSFB"",""2024-09-09"",""2024-09-22"","".FUNDACION SANTAFE DE BOGOTA""}]",7
"""MIFI-4506""","""PERIODO 202420 - 16 SEMANAS""","""VIRTUAL""","[{""-"","".VIRT"",""2024-08-05"",""2024-11-30"","".VIRTUAL"",""No info""}]",1


In [9]:
df = df.filter(pl.col('Campus') == "CAMPUS PRINCIPAL")
df = df.filter(pl.col('Parte de Periodo') != "DEPORTES")

### Usando los datos sin formatear

In [35]:
import pandas as pd

In [37]:
df = pd.read_json('../../scrap/no_formatted_cards_data.json')

In [38]:
df.sample(5)

,text
3076,IELE-1502 SISTEMAS DINÁMICOS\nProfesoresRequis...
848,CBIO-4912 TESIS MAESTRIA 1\nProfesoresRequisit...
1160,DCHO-1101 PROYECTOS 1 (TIPO E)\nProfesoresRequ...
3534,IQYA-3052 PROY. DESARROLLO PROFESIONAL IALI\nP...
2045,ECON-3311C COMPLEMENTARIA ECONOMETRÍA 1 CON TA...


In [39]:
df = df['text'].str.split('\n', expand=True)

In [40]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,ADMI-1101 FUNDAMENTOS DE ADMINISTRACION Y GERE...,ProfesoresRequisitos del curso,Cupo:,Sin cupo,NRC: 39342,Sección: 1,Créditos:3,Periodo: 202420,Parte de Periodo: 16 SEMANAS,Lugar: CAMPUS PRINCIPAL,...,None,None,None,None,None,None,None,None,None,None
1,ADMI-1101 FUNDAMENTOS DE ADMINISTRACION Y GERE...,ProfesoresRequisitos del curso,Cupo:,Sin cupo,NRC: 42986,Sección: 2,Créditos:3,Periodo: 202420,Parte de Periodo: 16 SEMANAS,Lugar: CAMPUS PRINCIPAL,...,None,None,None,None,None,None,None,None,None,None
2,ADMI-1101 FUNDAMENTOS DE ADMINISTRACION Y GERE...,ProfesoresRequisitos del curso,Cupo:,Sin cupo,NRC: 72305,Sección: 3,Créditos:3,Periodo: 202420,Parte de Periodo: 16 SEMANAS,Lugar: CAMPUS PRINCIPAL,...,None,None,None,None,None,None,None,None,None,None
3,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,ProfesoresRequisitos del curso,Cupo:,Sin cupo,NRC: 25454,Sección: 1,Créditos:3,Periodo: 202420,Parte de Periodo: 16 SEMANAS,Lugar: CAMPUS PRINCIPAL,...,None,None,None,None,None,None,None,None,None,None
4,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,ProfesoresRequisitos del curso,Cupo:,Sin cupo,NRC: 11411,Sección: 2,Créditos:3,Periodo: 202420,Parte de Periodo: 16 SEMANAS,Lugar: CAMPUS PRINCIPAL,...,None,None,None,None,None,None,None,None,None,None
